# Crypto Currency Exploration
# Data from RapidAPI
## Contact info:
### Sean Pharris - pharris.sean@gmail.com

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crypto-list/digital_currency_list.csv


API used for Crypto data is from RapidAPI - Alpha Vantage. https://rapidapi.com/alphavantage/api/alpha-vantage/
Documentation - https://www.alphavantage.co/documentation/

In [2]:
# import digital currency list (Contains all Crypto names and codes)
df = pd.read_csv('/kaggle/input/crypto-list/digital_currency_list.csv')

In [3]:
# creation of name list 
name_list ={'0-9':[],'A':[],'B':[],'C':[],'D':[],'E':[],'F':[],'G':[],'H':[],'I':[],'J':[],'K':[],'L':[],'M':[],'N':[],
            'O':[],'P':[],'Q':[],'R':[],'S':[],'T':[],'U':[],'V':[],'W':[],'X':[],'Y':[],'Z':[]}

for n, name in enumerate(df['currency name']):
    for i in name_list:
        if name[0] == i or name[0].upper() == i:
            name_list[i].append(name)